In [ ]:
pip install pgmpy

In [0]:
import numpy as np
import networkx as nx
import itertools
import pandas as pd
import joblib

In [0]:
#creating 100 graphs with nodes of the given structure
def graph_creation(nodes,graph_structure):
  W_matrix=[]
  b_matrix=[]
  for index in range(200):
    matrix=nx.to_numpy_array(graph_structure)
    W=np.random.normal(0.,1.,(nodes,nodes))
    W=(W+W.T)/2
    W*=matrix
    W_matrix.append(W)
    b_matrix.append(np.random.normal(0.,0.25,nodes)) #random generation
  return(W_matrix,b_matrix)




#creating the whole dataset for 5 nodes:
import math

def dataset(nodes):
  W_star,b_star=graph_creation(nodes,nx.star_graph(nodes-1))
  W_complete,b_complete=graph_creation(nodes,nx.complete_graph(nodes))
  W_BP,b_BP=graph_creation(nodes,nx.complete_bipartite_graph(math.ceil(nodes/2),nodes-math.ceil(nodes/2)))
  W_path,b_path=graph_creation(nodes,nx.path_graph(nodes))

  W_wheel,b_wheel=graph_creation(nodes,nx.wheel_graph(nodes))
  W_lollipop,b_lollipop=graph_creation(nodes,nx.lollipop_graph(math.ceil(nodes/2),nodes-math.ceil(nodes/2)))
  W_cycle,b_cycle=graph_creation(nodes,nx.cycle_graph(nodes))

  J=np.array(W_star+W_complete+W_BP+W_path+W_wheel+W_lollipop+W_cycle)
  b=np.array(b_star+b_complete+b_BP+b_path+b_path+b_wheel+b_lollipop+b_cycle)
  return (J,b)

J5_1400,b5_1400=dataset(5)
J10_1400,b10_1400=dataset(10)


joblib.dump(J5_1400,'/content/drive/My Drive/PGM/Project/J5_1400')
joblib.dump(J10_1400,'/content/drive/My Drive/PGM/Project/J10_1400')
joblib.dump(b5_1400,'/content/drive/My Drive/PGM/Project/b5_1400')
joblib.dump(b10_1400,'/content/drive/My Drive/PGM/Project/b10_1400')





In [0]:
import joblib
J5=joblib.load('/content/drive/My Drive/PGM/Project/J5_1400')
J10=joblib.load('/content/drive/My Drive/PGM/Project/J10_1400')

b5=joblib.load('/content/drive/My Drive/PGM/Project/b5_1400')
b10=joblib.load('/content/drive/My Drive/PGM/Project/b10_1400')

In [0]:
#Compute the marginals by enumeration exact inference
def generate_marginals(J, b):
  nodes = b.shape[1]
  target = []
  x_inputs = np.array(list(itertools.product(range(2), repeat=nodes)))
  #print(x_inputs)
  for J_i, b_i in zip(J, b):
    # print(J_i.shape, b_i.shape)
    target_i = []
    b_i.shape = b_i.shape[0], 1
    for x in x_inputs:
      x.shape = x.shape[0], 1
      prob_i = np.exp(b_i.T.dot(x) + x.T.dot(J_i).dot(x))
      target_i.append(prob_i[0,0])
    target_i = np.array(target_i)/np.sum(target_i)
    target.append(target_i)
  return np.array(target)

# this generates the total joint probability distribution. [00,01,10,11] -> [prob(00), prob(01), ...]

In [0]:
target5 = generate_marginals(J5, b5)
target10 = generate_marginals(J10, b10)


In [0]:
#calculating the marginals of each varible:
def variable_marginals(target,nodes):
  variable_marginal=[]
  for index in range(len(target)):
      marginal=[]
      x_inputs = np.array(list(itertools.product(range(2), repeat=nodes)))
      for var in range(nodes-1,-1,-1):
        indices=[i for i, x in enumerate(x_inputs) if x[var] == 0]
        temp=[sum(target[index][indices]),1-sum(target[index][indices])]
        marginal.append(temp)
      variable_marginal.append(np.array(marginal))
  return variable_marginal

true_marginal5=variable_marginals(target5,5)
true_marginal10=variable_marginals(target10,10)
#true_marginal15=variable_marginals(target15,15)
#true_marginal20=variable_marginals(target20,20)

# prob distribution of each variable. ex: prob(x1 = 0), prob(x1= 1)

In [ ]:
print(len(true_marginal5))

In [0]:
'''joblib.dump(true_marginal5,'/content/drive/My Drive/PGM/Project/true_marginal5_1400')
joblib.dump(true_marginal10,'/content/drive/My Drive/PGM/Project/true_marginal10_1400')'''



true_marginal5=joblib.load('/content/drive/My Drive/PGM/Project/true_marginal5_1400')
true_marginal10=joblib.load('/content/drive/My Drive/PGM/Project/true_marginal10_1400')




# Gated Graph Neural Network

In [0]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import networkx as nx
import itertools
import pickle
import time
import os
import joblib

In [0]:
path_to_pgm = "/content/drive/My Drive/PGM Project"
path_to_data = os.path.join(path_to_pgm, "Project")

In [0]:
J5=joblib.load(os.path.join(path_to_data, "J5"))
J10=joblib.load(os.path.join(path_to_data, "J10"))
J15=joblib.load(os.path.join(path_to_data, "J15"))


b5=joblib.load(os.path.join(path_to_data, "b5"))
b10=joblib.load(os.path.join(path_to_data, "b10"))
b15=joblib.load(os.path.join(path_to_data, "b15"))


targets5 = joblib.load(os.path.join(path_to_data, "true_marginal5"))
targets10 = joblib.load(os.path.join(path_to_data, "true_marginal10"))
targets15 = joblib.load(os.path.join(path_to_data, "true_marginal15"))

In [ ]:
targets5 = np.array(targets5)
targets10 = np.array(targets10)
targets15 = np.array(targets15)
print(targets5.shape, targets10.shape, targets15.shape)

In [0]:
class GatedGNN(nn.Module):
  def __init__(self,
               message_dim=5,
               hidden_state_dim=5,
               n_hidden_units_message=64,
               n_hidden_units_readout=64,
               n_timesteps=10
               ):
    super(GatedGNN, self).__init__()

    self.message_dim = message_dim
    self.hidden_state_dim = hidden_state_dim
    self.n_hidden_units_message = n_hidden_units_message
    self.n_hidden_units_readout = n_hidden_units_readout
    self.n_timesteps = n_timesteps

    # message passing MLP
    self.messageFunction = nn.Sequential(
        nn.Linear(2*hidden_state_dim+3, n_hidden_units_message),
        nn.ReLU(),
        nn.Linear(n_hidden_units_message, n_hidden_units_message),
        nn.ReLU(),
        nn.Linear(n_hidden_units_message, message_dim)
        )
    
    # GRU unit that updates the hidden states
    self.hiddenStateUpdateFunction = nn.GRUCell(message_dim, hidden_state_dim)

    # Readout MLP
    self.readoutFunction = nn.Sequential(
        nn.Linear(hidden_state_dim, n_hidden_units_readout),
        nn.ReLU(),
        nn.Linear(n_hidden_units_readout, n_hidden_units_readout),
        nn.ReLU(),
        nn.Linear(n_hidden_units_readout, 2)
        )
    
    self.initialize()
  
  def initialize(self):
    for m in self.modules():
      if isinstance(m, nn.Linear):
        m.weight.data.normal_(0, 0.1)
        m.bias.data.fill_(0)
  
  def forward(self, J, b):
    '''
      J: adjacency matrix tensor corresponding to a single graph. shape:(n,n)
      b: singletons tensor corresponding to a single graph. shape:(n,)
    '''
    n_nodes = J.shape[0]
    # messages = torch.zeros((n_nodes, self.message_dim))
    messages_node_i_node_j = torch.zeros((n_nodes, n_nodes, self.message_dim))
    hidden_states = torch.zeros((n_nodes, self.hidden_state_dim))

    for _ in range(self.n_timesteps):
      for node_i in range(n_nodes):
        for node_j in range(n_nodes):
          if J[node_i, node_j] != 0:
            input_to_message_function = torch.unsqueeze(
                torch.cat(
                    (
                        hidden_states[node_i], 
                        hidden_states[node_j], 
                        torch.tensor([J[node_i, node_j], b[node_i], b[node_j]])
                    ),
                    dim=0
                ),
                dim=0
            )
            message_from_node_i_to_node_j = self.messageFunction(input_to_message_function)
            # messages[node_j] += message_from_node_i_to_node_j[0]
            messages_node_i_node_j[node_i, node_j, :] = message_from_node_i_to_node_j[0]
      messages = torch.sum(messages_node_i_node_j,dim=0)
      hidden_states = self.hiddenStateUpdateFunction(messages, hidden_states)
    
    readout = self.readoutFunction(hidden_states)
    marginals = nn.Softmax(dim=1)(readout)
    # marginals = nn.Sigmoid()(readout)
    return marginals
  
  def train(self, J, b, targets, optimizer, loss_function, n_epochs=10, batch_size=50):
    '''
      optimizer: example :- Adam(GatedGNNobj.parameters(), lr=learning_rate).
      loss_function: example :- BinaryCrossEntropy() OR KLDivergenceLoss().
      n_epochs: no. of epochs.
      batch_size: no. of datapoints in a batch.
    '''
    # batch_loss_means = []
    for epoch_i in range(n_epochs):
      batch_losses = []
      count = 1
      batch_no = 0
      for J_i, b_i, targets_i in zip(J, b, targets):
        J_tensor = torch.from_numpy(J_i).float()
        b_tensor = torch.from_numpy(b_i).float()
        targets_tensor = torch.from_numpy(targets_i).float()

        marginals = self(J_tensor, b_tensor)
        loss_i = loss_function(marginals, targets_tensor)
        batch_losses.append(loss_i)
        
        if count%batch_size == 0:
          batch_no += 1
          self.zero_grad()
          
          batch_loss_mean = torch.stack(batch_losses).mean()
          batch_loss_mean.backward()
          
          optimizer.step()
          
          # batch_loss_means.append(batch_loss_mean.item())
          print("epoch "+str(epoch_i)+", batch "+str(batch_no)+" loss: ", batch_loss_mean.item())
          count = 0
          batch_losses = []
        
        count += 1
      if len(batch_losses) > 0:
        batch_no += 1
        self.zero_grad()
        batch_loss_mean = torch.stack(batch_losses).mean()
        batch_loss_mean.backward()
        optimizer.step()
        print("epoch "+str(epoch_i)+", batch "+str(batch_no)+" loss: ", batch_loss_mean.item())   
  
  def inference(self, J, b):
    '''
      J: adjacency matrices of graphs. shape: (N, nodes, nodes)
      b: singletons of graphs. shape: (N, nodes)
    '''
    marginals = []
    with torch.no_grad():
      for J_i, b_i in zip(J, b):
        J_tensor = torch.from_numpy(J_i).float()
        b_tensor = torch.from_numpy(b_i).float()
        marginals_i = self(J_tensor, b_tensor)
        marginals.append(marginals_i.detach().numpy())
    return np.array(marginals)
  
  def save_model(self, path):
    '''
      path: path to save the model.
    '''
    torch.save(self.state_dict(), path)



def load_model(model, path):
  '''
    model: GatedGNN object in which the saved model has to be loaded.
    path: path to the saved model.
  '''
  state_dict = torch.load(path, map_location=lambda storage,loc: storage)
  model.load_state_dict(state_dict)

class BinaryCrossEntropyLoss:
  def __init__(self):
    self.function = nn.BCELoss()
  
  def __call__(self, ypred, y):
    return self.function(ypred, y)

class KLDivergenceLoss:
  def __init__(self):
    self.function = nn.KLDivLoss()
  
  def __call__(self, ypred, y):
    ypred_log = torch.log(ypred)
    return self.function(ypred_log, y)


In [0]:
''' load model '''
ggnn = GatedGNN()
load_model(ggnn, os.path.join(path_to_pgm, "models/ggnn_model_J5"))

In [ ]:
start_time = time.time()
ypred = ggnn.inference(J5, b5)
kldloss = KL_loss(ypred, targets5)
# kldloss = KLDivergenceLoss()(torch.from_numpy(ypred).float(), torch.from_numpy(targets[-100:, :]).float())
print("kld loss: ", kldloss, np.sum(kldloss))
end_time = time.time()
print("time taken: ", (end_time-start_time)/60)

In [0]:
###########################################################
####################MCMC###################################

def samples(W,b,number_samples):
        samples=[]
        X = np.array([1 if np.random.rand() < .5 else -1 for i in range(len(b))])
        #print(X.shape)
        sample = [np.copy(X)]
        for i in range(5000):
            for j in range(len(b)):
                t = W[j,:].dot(X)+b[j]
                conditional_prob=1./(1.+np.exp(-t))
                X[j] = +1 if np.random.rand() < conditional_prob else -1
            sample.append(np.copy(X))
        index=np.random.randint(0,5,number_samples)
        sample=pd.DataFrame(sample)
        samples.append(np.array(sample.iloc[index]))

        return samples

def MCMC(graph_samples):
    result = []
    for samples in graph_samples:
        sample01 = np.where(samples > 0,0,1)
        sample01 = np.array(sample01)
        #print(sample01)
        positive = sample01.mean(axis=0)
        result.append(np.stack([1-positive, positive], axis=1))
    return result


def prediction(J,b):
  predicted_MCMC=[]
  for index in range(len(J)):
    graph_samples=samples(J[index],b[index],number_samples=1000)
    #print(graph_samples)
    predicted_MCMC+=(MCMC(graph_samples))
  return predicted_MCMC

#calculation of time for different number of nodes with same number of graphs




In [0]:
#KL Loss for MCMC algorithm:
def KL_loss(p, q):
  result=0
  for i in range(len(p)):
    if q[i].all()!=0:
	    result+=((p[i] * np.log(p[i]/q[i])))
    else:
      result+=((p[i] * np.log(p[i]/0.001)))
  #print(result)
  return sum(result)/len(p)

In [0]:
import time


startTime=time.time()
predicted_MCMC5=prediction(J5,b5)
endTime=time.time()
print("The Time Required For 5 nodes graph is (in seconds)",endTime-startTime)


startTime=time.time()
predicted_MCMC10=prediction(J10,b10)
endTime=time.time()
print("The Time Required For 10 nodes graph is (in seconds)",endTime-startTime)



startTime=time.time()
predicted_MCMC15=prediction(J15,b15)
endTime=time.time()
print("The Time Required For 15 nodes graph is (in seconds)",endTime-startTime)


startTime=time.time()
predicted_MCMC20=prediction(J20,b20)
endTime=time.time()
print("The Time Required For 20 nodes graph is (in seconds)",endTime-startTime)




print("Loss for graphs with nodes 5 is ",sum(KL_loss(true_marginal5,predicted_MCMC5)))
print("Loss for graphs with nodes 10 is ",sum(KL_loss(true_marginal10,predicted_MCMC10)))
#print("Loss for graphs with nodes 15 is ",sum(KL_loss(true_marginal15,predicted_MCMC15)))

In [ ]:
##############################BP########################
### code is written only for star graph with nodes 5 else changes are made for marking the observations


from pgmpy.models import FactorGraph
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.inference import BeliefPropagation
def star_factor_graph(J,b):
  prediction=' '
  G = FactorGraph()
  G.add_nodes_from(['x1', 'x2', 'x3','x4','x5'])
  f1 = DiscreteFactor(['x1', 'x2'], [2, 2], np.exp(np.array([[0,0],[0,J[0][1]]])))
  f2 = DiscreteFactor(['x1', 'x3'], [2, 2], np.exp(np.array([[0,0],[0,J[0][2]]])))
  f3 = DiscreteFactor(['x1', 'x4'], [2, 2], np.exp(np.array([[0,0],[0,J[0][3]]])))
  f4 = DiscreteFactor(['x1', 'x5'], [2, 2], np.exp(np.array([[0,0],[0,J[0][4]]])))
  f5 = DiscreteFactor(['x1'], [2], np.exp(np.array([[0,b[0]]])))
  f6 = DiscreteFactor(['x2'], [2], np.exp(np.array([[0,b[1]]])))
  f7 = DiscreteFactor(['x3'], [2], np.exp(np.array([[0,b[2]]])))
  f8 = DiscreteFactor(['x4'], [2], np.exp(np.array([[0,b[3]]])))
  f9 = DiscreteFactor(['x5'], [2], np.exp(np.array([[0,b[4]]])))
  G.add_factors(f1,f2,f3,f4,f5,f6,f7,f8,f9)
  G.add_edges_from([('x1',f1),('x1',f2),('x1',f3),('x1',f4),(f1,'x2'),(f2,'x3'),(f3,'x4'),(f4,'x5'),(f5,'x1'),(f6,'x2'),(f7,'x3'),(f8,'x4'),(f9,'x5')])
  k=BeliefPropagation(G)  
  return k                      
 
def predict(bp):
  predict=[]
  predict+=[str(bp.query(['x1']))[79:85],str(bp.query(['x1']))[123:129]]
  predict+=[str(bp.query(['x2']))[79:85],str(bp.query(['x2']))[123:129]]
  predict+=[str(bp.query(['x3']))[79:85],str(bp.query(['x3']))[123:129]]
  predict+=[str(bp.query(['x4']))[79:85],str(bp.query(['x4']))[123:129]]
  predict+=[str(bp.query(['x5']))[79:85],str(bp.query(['x5']))[123:129]]
  return predict

def prediction_values(J,b):
  result=[]
  for index in range(len(J)):
    bp=star_factor_graph(J[index],b[index])
    prediction=predict(bp)
    prediction=[float(x) for x in prediction]
    sum_var=sum(prediction[0:2])
    for index in range(0,10,2):
      prob=[i/sum_var for i in prediction]
    split=[2,4,6,8]
    prob=[prob[i : j] for i, j in zip([0] + split , split + [None])]  
    result.append(np.array(prob))
  return result


import time

startTime=time.time()
predicted_star5BP= prediction_values(J5[:50],b5[:50])
endTime=time.time()
print("The Time Required For 5 nodes star graph is (in seconds)",endTime-startTime)  
print("The KL Loss for 5 nodes star graph is ",sum(KL_loss(true_marginal5[:50],predicted_star5BP)))